In [24]:
import json

file = json.load(open("data_.json", "r"))
print(file[2]["output"])

[  {    "date": "2024-11-18",    "patch_name": "Alert for CVE-2024-21287",    "link": "https://www.oracle.com/in/security-alerts/alert-cve-2024-21287.html"  },  {    "date": "2022-05-25",    "patch_name": "Alert for CVE-2022-21500",    "link": "https://www.oracle.com/in/security-alerts/alert-cve-2022-21500.html"  },  {    "date": "2021-12-17",    "patch_name": "Alert for CVE-2021-44228",    "link": "https://www.oracle.com/in/security-alerts/alert-cve-2021-44228.html"  },  {    "date": "2020-11-06",    "patch_name": "Alert for CVE-2020-14750",    "link": "https://www.oracle.com/in/security-alerts/alert-cve-2020-14750.html"  },  {    "date": "2025-07-",    "patch_name": "Solaris Third Party Bulletin - July 2025",    "link": "https://www.oracle.com/in/security-alerts/bulletinjul2025.html"  }]


In [2]:
!pip uninstall -y unsloth peft

!pip install unsloth trl peft accelerate bitsandbytes

Found existing installation: peft 0.17.0
Uninstalling peft-0.17.0:
  Successfully uninstalled peft-0.17.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 3.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.7/182.7 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [3]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: Tesla T4


In [4]:
import unsloth

unsloth??

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
unsloth.DEVICE_TYPE

'cuda'

In [6]:
# dir(unsloth)

In [25]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.8.6: Fast Mistral patching. Transformers: 4.55.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [26]:
from datasets import Dataset

def format_prompt(example):
    return f"### Input: {example['input']}\n### Output: {json.dumps(example['output'])}<|endoftext|>"

formatted_data = [format_prompt(item) for item in file]
dataset = Dataset.from_dict({"text": formatted_data})

# dataset['text']
dataset

Dataset({
    features: ['text'],
    num_rows: 11
})

In [27]:
dir(FastLanguageModel)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'for_inference',
 'for_training',
 'from_pretrained',
 'get_peft_model',
 'patch_peft_model',
 'post_patch',
 'pre_patch']

In [28]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

In [29]:
  # max_seq_length = 1024

In [30]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=2,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=4,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=2,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/11 [00:00<?, ? examples/s]

In [31]:
# Train the model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 11 | Num Epochs = 4 | Total steps = 24
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 2 x 1) = 2
 "-____-"     Trainable parameters = 119,537,664 of 3,940,617,216 (3.03% trained)


Step,Training Loss
2,1.086200
4,0.962500
6,1.098100
8,1.031100
10,0.729200
12,0.575100
14,0.614500
16,0.579800
18,0.423500
20,0.301900


In [32]:
trainer_stats

TrainOutput(global_step=24, training_loss=0.6793431515494982, metrics={'train_runtime': 204.782, 'train_samples_per_second': 0.215, 'train_steps_per_second': 0.117, 'total_flos': 975313616707584.0, 'train_loss': 0.6793431515494982})

In [33]:
# Test the fine-tuned model
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Test prompt
messages = [
    {"role": "user", "content": "Extract patches from givr text : 7 min read Streamlabs Desktop Patch Notes (https://streamlabs.com/content-hub/post/streamlabs-desktop-patch-notes) Find out what's new and improved in Streamlabs Desktop with our patch notes. April 22, 2024 Christopher Meyers (https://streamlabs.com/content-hub/author/cmeyers) Streamlabs Desktop (https://streamlabs.com/content-hub/streamlabs-desktop) Patch Notes (https://streamlabs.com/content-hub/streamlabs-desktop/patch-notes) 7 min read Streamlabs Desktop Patch Notes (https://streamlabs.com/content-hub/post/streamlabs-desktop-patch-notes) Find out what's new and improved in Streamlabs Desktop with our patch notes. April 22, 2024 Christopher Meyers (https://streamlabs.com/content-hub/author/cmeyers) Get Streamlabs Desktop — go live in minutes! Free Windows 240MB Free macOS 10.14+ 309MB Streaming (https://streamlabs.com/content-hub/streaming) Getting Started (https://streamlabs.com/content-hub/streaming/getting-started) Growing your audience (https://streamlabs.com/content-hub/streaming/growing-your-audience) Hardware (https://streamlabs.com/content-hub/streaming/hardware) Monetization (https://streamlabs.com/content-hub/streaming/monetization) Mobile (https://streamlabs.com/content-hub/streaming/mobile) Charity (https://streamlabs.com/content-hub/streaming/charity) Gaming (https://streamlabs.com/content-hub/streaming/gaming) Live Events (https://streamlabs.com/content-hub/streaming/live-events) News (https://streamlabs.com/content-hub/streaming/news) MentalHealth (https://streamlabs.com/content-hub/streaming/mental-health) Creator Spotlights (https://streamlabs.com/content-hub/streaming/creator-spotlights) Platforms (https://streamlabs.com/content-hub/platforms) Instagram (https://streamlabs.com/content-hub/platforms/instagram) X (Twitter) (https://streamlabs.com/content-hub/platforms/x-twitter) Twitch (https://streamlabs.com/content-hub/platforms/twitch) YouTube (https://streamlabs.com/content-hub/platforms/youtube) Facebook (https://streamlabs.com/content-hub/platforms/facebook) Trovo (https://streamlabs.com/content-hub/platforms/trovo) Other Platforms (https://streamlabs.com/content-hub/platforms/other-platforms) Kick (https://streamlabs.com/content-hub/platforms/kick) Desktop (https://streamlabs.com/content-hub/streamlabs-desktop) Setup (https://streamlabs.com/content-hub/streamlabs-desktop/setup) Stream Overlays (https://streamlabs.com/content-hub/streamlabs-desktop/stream-overlays) App Store (https://streamlabs.com/content-hub/streamlabs-desktop/app-store) Widgets (https://streamlabs.com/content-hub/streamlabs-desktop/streamlabs-desktop-widgets) Other (Streamlabs Desktop) (https://streamlabs.com/content-hub/streamlabs-desktop/streamlabs-desktop-other) Patch Notes (https://streamlabs.com/content-hub/streamlabs-desktop/patch-notes) Tools (https://streamlabs.com/content-hub/tools) Ultra (https://streamlabs.com/content-hub/tools/ultra) Stream Overlays (https://streamlabs.com/content-hub/tools/tools-stream-overlays) Mobile App (https://streamlabs.com/content-hub/tools/mobile-app) Tips (https://streamlabs.com/content-hub/tools/tips) Cloudbot (https://streamlabs.com/content-hub/tools/cloudbot) App Store (https://streamlabs.com/content-hub/tools/tools-app-store) Merch (https://streamlabs.com/content-hub/tools/merch) Charity (https://streamlabs.com/content-hub/tools/tools-charity) Other Tools (https://streamlabs.com/content-hub/tools/other-tools) Streamlabs Console (https://streamlabs.com/content-hub/tools/streamlabs-console) Cross Clip (https://streamlabs.com/content-hub/tools/cross-clip) Video Editor (https://streamlabs.com/content-hub/tools/video-editor) Talk Studio (https://streamlabs.com/content-hub/tools/talk-studio) Podcast Editor (https://streamlabs.com/content-hub/tools/podcast-editor) Link Space (https://streamlabs.com/content-hub/tools/link-space) Widgets (https://streamlabs.com/content-hub/widgets) Alert Box (https://streamlabs.com/content-hub/widgets/alert-box) Event List (https://streamlabs.com/content-hub/widgets/event-list) Chat Box (https://streamlabs.com/content-hub/widgets/chat-box) Stream Labels (https://streamlabs.com/content-hub/widgets/stream-labels)"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

# Generate response
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=2056,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

# Decode and print
response = tokenizer.batch_decode(outputs)[0]
print(response)

<|user|> Extract patches from givr text : 7 min read Streamlabs Desktop Patch Notes (https://streamlabs.com/content-hub/post/streamlabs-desktop-patch-notes) Find out what's new and improved in Streamlabs Desktop with our patch notes. April 22, 2024 Christopher Meyers (https://streamlabs.com/content-hub/author/cmeyers) Streamlabs Desktop (https://streamlabs.com/content-hub/streamlabs-desktop) Patch Notes (https://streamlabs.com/content-hub/streamlabs-desktop/patch-notes) 7 min read Streamlabs Desktop Patch Notes (https://streamlabs.com/content-hub/post/streamlabs-desktop-patch-notes) Find out what's new and improved in Streamlabs Desktop with our patch notes. April 22, 2024 Christopher Meyers (https://streamlabs.com/content-hub/author/cmeyers) Get Streamlabs Desktop — go live in minutes! Free Windows 240MB Free macOS 10.14+ 309MB Streaming (https://streamlabs.com/content-hub/streaming) Getting Started (https://streamlabs.com/content-hub/streaming/getting-started) Growing your audience (

In [39]:
# Test the fine-tuned model
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Test prompt
messages = [
    {"role": "user", "content": "Extract patches from givr text : with Oracle Solaris. Solaris Third Party Bulletins are published on the same day as Oracle Critical Patch Updates are released. These bulletins will be updated on the third Tuesday of the following two months after their release (i.e., the two months between the normal quarterly Critical Patch Update publication dates). In addition, Solaris Third Party Bulletins may also be updated for vulnerability patches deemed too critical to wait for the next scheduled publication date. Solaris Third Party Bulletins released before 2020 are available here (/in/security-alerts/solarisbulletinsarchive.html) . Solaris Third Party Bulletin Latest Version/Date Solaris Third Party Bulletin - July 2025 (/in/security-alerts/bulletinjul2025.html)Rev 1, 15 July 2025 Solaris Third Party Bulletin - April 2025 (/in/security-alerts/bulletinapr2025.html) Rev 3, 18 June 2025 Solaris Third Party Bulletin - January 2025 (/in/security-alerts/bulletinjan2025.html) Rev 3, 18 March 2025 Solaris Third Party Bulletin - October 2024 (/in/security-alerts/bulletinoct2024.html) Rev 4, 07 May 2025 Solaris Third Party Bulletin - July 2024 (/in/security-alerts/bulletinjul2024.html) Rev 3, 24 September 2024 Solaris Third Party Bulletin - April 2024 (/in/security-alerts/bulletinapr2024.html) Rev 3, 14 June 2024 Solaris Third Party Bulletin - January 2024 (/in/security-alerts/bulletinjan2024.html) Rev 3, 19 March 2024 Solaris Third Party Bulletin - October 2023 (/in/security-alerts/bulletinoct2023.html) Rev 3, 19 December 2023 Solaris Third Party Bulletin"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

# Generate response
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=1024,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

# Decode and print
response = tokenizer.batch_decode(outputs)[0]
in_ = response.index("<|assistant|>")
print(response[in_:])

<|assistant|> "[  {    \"date\": \"2025-07-15\",    \"patch_name\": \"Solaris Third Party Bulletin - July 2025\",    \"link\": \"https://www.oracle.com/in/security-alerts/bulletinjul2025.html\"  },  {    \"date\": \"2025-06-18\",    \"patch_name\": \"Solaris Third Party Bulletin - April 2025\",    \"link\": \"https://www.oracle.com/in/security-alerts/bulletinapr2025.html\"  },  {    \"date\": \"2025-03-18\",    \"patch_name\": \"Solaris Third Party Bulletin - January 2025\",    \"link\": \"https://www.oracle.com/in/security-alerts/bulletinjan2025.html\"  },  {    \"date\": \"2025-05-07\",    \"patch_name\": \"Solaris Third Party Bulletin - October 2024\",    \"link\": \"https://www.oracle.com/in/security-alerts/bulletinoct2024.html\"  },  {    \"date\": \"2024-09-24\",    \"patch_name\": \"Solaris Third Party Bulletin - July 2024\",    \"link\": \"https://www.oracle.com/in/security-alerts/bulletinjul2024.html\"  },  {    \"date\": \"2024-06-14\",    \"patch_name\": \"Solaris Third Part

In [ ]:
model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")

In [ ]:
gguf_files = [f for f in os.listdir("gguf_model") if f.endswith(".gguf")]
if gguf_files:
    gguf_file = os.path.join("gguf_model", gguf_files[0])
    print(f"Downloading: {gguf_file}")
    files.download(gguf_file)